In [ ]:
import praw
import pandas as pd
import datetime

import xlsxwriter

import requests
import urllib
from tqdm import tqdm
import time
import os

In [ ]:
def get_date(submission):
    time = submission.created
    return datetime.datetime.fromtimestamp(time)

In [ ]:
def save_image(url):
    
    file_name = url.split("/")[-1]
    res = requests.get(url, stream = True)

    # Creating an images folder if it doens't already exist
    if not os.path.exists('images'):
       os.makedirs('images')

    urllib.request.urlretrieve(url, "images/" + file_name)

In [ ]:
reddit_read_only = praw.Reddit(
    client_id='JzbofoB1-Xrh3gbv--LVTQ',
    client_secret='v3R81WmJnSJJ8WWdjn4sZvRcerIVGg',
    user_agent='Ukraine Scraper'
)
# qwerrtty423499 username
# KVKb6AqeabAG2PD password
# cveodyiphyxqccujnc@tmmbt.com email
# v3R81WmJnSJJ8WWdjn4sZvRcerIVGg secret

subreddit = reddit_read_only.subreddit("memes")

# posts = subreddit.hot(limit=10)
limit = 100
posts = subreddit.top(time_filter='year', limit=limit)
# Scraping the top posts of the current month



posts_dict = {"Title": [], "Author" : [], "Timestamp" : [], "Post Text": [],
            "ID": [], "Score": [],
            "Total Comments": [], "Image URL": []
            }

with tqdm(total=limit, bar_format='{l_bar}{bar:50}{r_bar}{bar:-10b}') as progress_bar:
    for post in posts:
        # URL of each post
        # ensures posts without images are ignored
        if (".jpg" in post.url or ".png" in post.url or ".tiff" in post.url or ".gif" in post.url):
            # image is valid
            posts_dict["Image URL"].append(str(post.url))
            save_image(str(post.url))

        else:
            continue 

        # Title of each post
        posts_dict["Title"].append(post.title)

        # Author of each post
        posts_dict["Author"].append(str(post.author))

        # Time post was created
        posts_dict["Timestamp"].append(datetime.datetime.fromtimestamp(post.created).strftime("%m/%d/%Y, %H:%M:%S"))

        # Text inside a post
        posts_dict["Post Text"].append(post.selftext)

        # Unique ID of each post
        posts_dict["ID"].append(post.id)

        # The score of a post
        posts_dict["Score"].append(post.score)

        # Total number of comments inside the post
        posts_dict["Total Comments"].append(post.num_comments)
        
        progress_bar.update()


# Displaying the data in a pandas dataframe (for testing purposes)
# pd.set_option('display.max_rows', 100)
# pd.set_option('max_colwidth', None)

# top_posts = pd.DataFrame(posts_dict)
# top_posts



In [ ]:
# wb = Workbook()
# sheet1 = wb.add_sheet('Reddit Data 1')
workbook = xlsxwriter.Workbook('Expenses01.xlsx')
worksheet = workbook.add_worksheet()

# column, row

# Writing the table headers
i = 0
for key in posts_dict:
    worksheet.write(0, i, key)
    i+=1

# Writing the table data
for x in range(1, len(posts_dict["Title"])):
    i = 0
    for key in posts_dict:
        if (key == "Image URL"):
            worksheet.write_url(x, i, posts_dict[key][x])
        else:
            worksheet.write(x, i, posts_dict[key][x])
        i+=1

# Saving the document
workbook.close()
